In [3]:
1-1

0

In [4]:
using NearestNeighbors
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Base.Collections
using WordStreams
using DataStructures
using Clustering
using Distances
using Iterators
using StatsBase
using Training

In [5]:
ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model");
dtree,labels = nn_tree(ee)
""

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in slice(::Array{Float32,2}, ::Vararg{Any,N}) at ./deprecated.jl:30
 in evaluate at /home/ubuntu/.julia/v0.5/NearestNeighbors/src/evaluation.jl:47 [inlined] (repeats 2 times)
 in create_bsphere(::Array{Float32,2}, ::Query.AngularDist, ::Array{Int64,1}, ::Int64, ::Int64) at /home/ubuntu/.julia/v0.5/NearestNeighbors/src/hyperspheres.jl:58
 in build_BallTree(::Int64, ::Array{Float32,2}, ::Array{Float32,2}, ::Array{NearestNeighbors

In [27]:
@time nears=find_nearest_words(ee,"states",dtree,labels )

1386.692237 seconds (256.42 M allocations: 37.216 GB, 1.85% gc time)


Array{Tuple{Tuple{String,Int64},Float32},1}[Tuple{Tuple{String,Int64},Float32}[(("states",1),0.000155425),(("possibility",15),0.267356),(("initially",10),0.267787),(("kingship",1),0.270334),(("rule",20),0.276474)],Tuple{Tuple{String,Int64},Float32}[(("states",2),0.0),(("strategy",20),0.255762),(("map",17),0.270634),(("united",5),0.273032),(("directors",6),0.278503)],Tuple{Tuple{String,Int64},Float32}[(("states",3),0.0),(("february",20),0.336462),(("opened",16),0.340318),(("outselling",1),0.340817),(("1992",5),0.342176)],Tuple{Tuple{String,Int64},Float32}[(("states",4),0.0),(("line",4),0.265136),(("billion",16),0.266845),(("households",18),0.273618),(("3",3),0.274329)],Tuple{Tuple{String,Int64},Float32}[(("states",5),0.0),(("germany",2),0.282167),(("prussia's",1),0.316669),(("relations",5),0.317873),(("annexation",1),0.317923)],Tuple{Tuple{String,Int64},Float32}[(("states",6),0.0),(("assist",15),0.242816),(("all",2),0.24366),(("review",6),0.24379),(("taken",14),0.244824)],Tuple{Tuple{St

In [28]:
trained_times = collect(values(ee.trained_times["states"]))
items = sort(collect(zip(trained_times, nears)))
map(println, items)

(3179,Tuple{Tuple{String,Int64},Float32}[(("states",15),0.000155425),(("united",17),0.169622),(("bill",9),0.188513),(("clinton",8),0.192871),(("confirmed",16),0.20403)])
(3273,Tuple{Tuple{String,Int64},Float32}[(("states",12),0.0),(("1940",2),0.252151),(("1937",11),0.255052),(("biography",12),0.260157),(("woodrow",1),0.260285)])
(3320,Tuple{Tuple{String,Int64},Float32}[(("states",3),0.0),(("february",20),0.336462),(("opened",16),0.340318),(("outselling",1),0.340817),(("1992",5),0.342176)])
(3515,Tuple{Tuple{String,Int64},Float32}[(("states",4),0.0),(("line",4),0.265136),(("billion",16),0.266845),(("households",18),0.273618),(("3",3),0.274329)])
(3616,Tuple{Tuple{String,Int64},Float32}[(("states",17),0.0),(("private",18),0.196623),(("navy",3),0.200308),(("boatswain's",1),0.205103),(("reserve",7),0.209955)])
(3832,Tuple{Tuple{String,Int64},Float32}[(("states",6),0.0),(("assist",15),0.242816),(("all",2),0.24366),(("review",6),0.24379),(("taken",14),0.244824)])
(4210,Tuple{Tuple{String,Int

Void[nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing,nothing]

In [31]:
function load_wordsim353(filename="data/corpora/wordsim353/combined.csv")
    wordsims = readdlm(filename, ','; skipstart=1)
    wordpairs = convert(Matrix{String},wordsims[:,1:2])
    groundsim = convert(Vector{Float64},wordsims[:,3])
    (wordpairs, groundsim)
end
wordpairs, groundsim = load_wordsim353()

(
String["love" "sex"; "tiger" "cat"; "tiger" "tiger"; "book" "paper"; "computer" "keyboard"; "computer" "internet"; "plane" "car"; "train" "car"; "telephone" "communication"; "television" "radio"; "media" "radio"; "drug" "abuse"; "bread" "butter"; "cucumber" "potato"; "doctor" "nurse"; "professor" "doctor"; "student" "professor"; "smart" "student"; "smart" "stupid"; "company" "stock"; "stock" "market"; "stock" "phone"; "stock" "CD"; "stock" "jaguar"; "stock" "egg"; "fertility" "egg"; "stock" "live"; "stock" "life"; "book" "library"; "bank" "money"; "wood" "forest"; "money" "cash"; "professor" "cucumber"; "king" "cabbage"; "king" "queen"; "king" "rook"; "bishop" "rabbi"; "Jerusalem" "Israel"; "Jerusalem" "Palestinian"; "holy" "sex"; "fuck" "sex"; "Maradona" "football"; "football" "soccer"; "football" "basketball"; "football" "tennis"; "tennis" "racket"; "Arafat" "peace"; "Arafat" "terror"; "Arafat" "Jackson"; "law" "lawyer"; "movie" "star"; "movie" "popcorn"; "movie" "critic"; "movie" 

In [ ]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String})
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            wvs_1 = hcat(ee.embedding[wordpairs[ii,1]]...)
            wvs_2 = hcat(ee.embedding[wordpairs[ii,2]]...)
            sims[ii]=1-minimum(pairwise(CosineDist(), wvs_1,wvs_2))
        catch ex
            warn(ex)
            sims[ii]=NaN
        end
    end
    return sims
end
sims = get_sims(ee,map(lowercase,wordpairs))
corspearman(groundsim,sims)

2016-07-05T19:30:21.263 - warn: KeyError: key "aglow" not found prefix: "error: "
2016-07-05T19:30:22.882 - warn: KeyError: key "angstrom" not found prefix: "error: "
2016-07-05T19:30:23.53 - warn: KeyError: key "litigate" not found prefix: "error: "
2016-07-05T19:30:23.695 - warn: KeyError: key "arpeggio" not found prefix: "error: "
2016-07-05T19:30:24.257 - warn: KeyError: key "recombine" not found prefix: "error: "
2016-07-05T19:30:24.779 - warn: KeyError: key "gherkin" not found prefix: "error: "
2016-07-05T19:30:24.945 - warn: KeyError: key "smother" not found prefix: "error: "
2016-07-05T19:30:25.452 - warn: KeyError: key "qibla" not found prefix: "error: "
2016-07-05T19:30:25.454 - warn: KeyError: key "anorgasmia" not found prefix: "error: "
2016-07-05T19:30:25.847 - warn: KeyError: key "neckline" not found prefix: "error: "
2016-07-05T19:30:26.196 - warn: KeyError: key "backdate" not found prefix: "error: "
2016-07-05T19:30:26.218 - warn: KeyError: key "repercussion" not found 

0.36608622781802247

In [34]:
function load_SCWS(filename = "data/corpora/SCWS/ratings.txt")
    function get_context2(fulltext)
        Task() do
            skip_next = 0
            for word in split(fulltext, ' ')
                skip_next-=1
                if word == "<b>"
                    skip_next=3 
                end
                skip_next>0 && continue
                produce(lowercase(word))
            end
        end |> collect
    end
    entries = readdlm(filename, '\t', String; quotes=false, comments=false)
    wordpairs = map(lowercase,entries[:,[2,4]])
    groundsim = map(s->parse(Float64,s), entries[:,8])
    contexts = map(get_context2,entries[:,[6,7]])
    (wordpairs,contexts,groundsim,)
end
wordpairs,contexts,groundsim = load_SCWS();
""

""

In [ ]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String}, contexts::Matrix)
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            
            sense1 = Training.WSD(ee, wordpairs[ii,1],contexts[ii,1];skip_oov=true)
            wv1 = ee.embedding[wordpairs[ii,1]][sense1]
            
            sense2 = Training.WSD(ee, wordpairs[ii,2],contexts[ii,2];skip_oov=true)
            wv2 = ee.embedding[wordpairs[ii,2]][sense2]
            
            sims[ii]=1-cosine_dist(wv1,wv2)
        catch ex
            if typeof(ex)==KeyError
                warn(ex)
                sims[ii]=NaN
            else
                rethrow(ex)
            end
        end
    end
    return sims
end
local_sims = get_sims(ee,wordpairs,contexts)
corspearman(groundsim,local_sims)

2016-07-05T19:29:09.719 - warn: KeyError: key "aglow" not found prefix: "error: "
2016-07-05T19:29:12.812 - warn: KeyError: key "tip-off" not found prefix: "error: "
2016-07-05T19:29:13.186 - warn: KeyError: key "exhale" not found prefix: "error: "
2016-07-05T19:29:13.187 - warn: KeyError: key "insufflate" not found prefix: "error: "
2016-07-05T19:29:13.546 - warn: KeyError: key "angstrom" not found prefix: "error: "
2016-07-05T19:29:14.548 - warn: KeyError: key "litigate" not found prefix: "error: "
2016-07-05T19:29:14.87 - warn: KeyError: key "arpeggio" not found prefix: "error: "
2016-07-05T19:29:15.871 - warn: KeyError: key "recombine" not found prefix: "error: "
2016-07-05T19:29:16.763 - warn: KeyError: key "gherkin" not found prefix: "error: "
2016-07-05T19:29:17.054 - warn: KeyError: key "smother" not found prefix: "error: "
2016-07-05T19:29:17.954 - warn: KeyError: key "qibla" not found prefix: "error: "
2016-07-05T19:29:17.956 - warn: KeyError: key "anorgasmia" not found prefi

0.33274357219353984

In [ ]:

function weighted_average(logprobs, embeddings)
    function normal_probs(logprobs::Vector)
        ret = copy(logprobs)
        max_lp = maximum(logprobs)
        ret.-=max_lp #Bring closer to zero
        map!(exp,ret)
        denom = sum(ret)
        ret./=denom
    end
    
    
    ret = zeros(size(embeddings[1]))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight*embedding
    end
    ret
end

function all_logprobs_of_context(embed, context, middle_word)
    [Query.logprob_of_context(embed, context,wv; skip_oov=true, normalise_over_length=true)
        for wv in embed.embedding[middle_word]]
end

function synthesize_embedding(ee,context, middle_word)
    logprobs = all_logprobs_of_context(ee, context,middle_word)
    weighted_average(logprobs, ee.embedding[middle_word])
end

function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String}, contexts::Matrix)
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try           
            wv1 = synthesize_embedding(ee, contexts[ii,1],wordpairs[ii,1])
            wv2 = synthesize_embedding(ee, contexts[ii,2],wordpairs[ii,2])
             
            
            sims[ii]=1-cosine_dist(wv1,wv2)
        catch ex
            if typeof(ex)==KeyError
                warn(ex)
                sims[ii]=NaN
            else
                rethrow(ex)
            end
        end
    end
    return sims
end
local_sims = get_sims(ee,wordpairs,contexts)
corspearman(groundsim,local_sims)

2016-07-05T19:48:51.151 - warn: KeyError: key "aglow" not found prefix: "error: "
2016-07-05T19:48:53.342 - warn: KeyError: key "tip-off" not found prefix: "error: "
2016-07-05T19:48:53.752 - warn: KeyError: key "exhale" not found prefix: "error: "
2016-07-05T19:48:53.755 - warn: KeyError: key "insufflate" not found prefix: "error: "
2016-07-05T19:48:54.154 - warn: KeyError: key "angstrom" not found prefix: "error: "
2016-07-05T19:48:55.249 - warn: KeyError: key "litigate" not found prefix: "error: "
2016-07-05T19:48:55.542 - warn: KeyError: key "arpeggio" not found prefix: "error: "
2016-07-05T19:48:56.569 - warn: KeyError: key "recombine" not found prefix: "error: "
2016-07-05T19:48:57.469 - warn: KeyError: key "gherkin" not found prefix: "error: "
2016-07-05T19:48:57.754 - warn: KeyError: key "smother" not found prefix: "error: "
2016-07-05T19:48:58.633 - warn: KeyError: key "qibla" not found prefix: "error: "
2016-07-05T19:48:58.636 - warn: KeyError: key "anorgasmia" not found pref

0.5111245622908596

In [40]:
?Query.logprob_of_context

No documentation found.

`Query.logprob_of_context` is a `Function`.

```
# 2 methods for generic function "logprob_of_context":
logprob_of_context{S<:String}(embed::WordEmbeddings.WordEmbedding, context::AbstractArray{S,1}, middle_word::S) at /mnt_volume/phd/prototypes/SenseSplittingWord2Vec/src/Query.jl:142
logprob_of_context{S<:String}(embed::WordEmbeddings.GenWordEmbedding, context::AbstractArray{S,1}, input::Array{Float32,1}; skip_oov, normalise_over_length) at /mnt_volume/phd/prototypes/SenseSplittingWord2Vec/src/Query.jl:148
```


  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [45]:
describe(collect(values(ee.distribution)))

Summary Stats:
Mean:         0.000011
Minimum:      0.000000
1st Quartile: 0.000000
Median:       0.000001
3rd Quartile: 0.000003
Maximum:      0.019961
